<a href="https://colab.research.google.com/github/AnSe2002/Project-workshop-on-the-development-of-ETL-solutions/blob/main/vebinar/%D0%92%D0%B8%D0%B7%D1%83%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%B4%D0%BB%D1%8F_%D0%B2%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Загрузка данных.

In [1]:
import altair as alt
import pandas as pd

df = pd.read_excel('1.3.xlsx')

Гистограмма распределения возраста

In [4]:
hist_age = alt.Chart(df).mark_bar().encode(
    alt.X("age:Q", bin=True, title="Возраст"),
    y='count()',
    tooltip=[alt.Tooltip('count()', title='Количество')]
).properties(
    title="Распределение возраста"
)

hist_age

alt.Chart(...)

Больше всего людей (17 человек) в диапазоне от 40 до 50 лет, а реже всего встречаются от 10 до 20 лет, всего 4 человека.

Расчёт топа 5 по городам.

In [6]:
# Извлечение города из адреса
df["city"] = df["address"].str.split(",").str[0]

# Теперь можно выполнить анализ топ-5 городов
top_cities = df["city"].value_counts().head(5)
print("Топ-5 городов по количеству проживающих:")
print(top_cities)

Топ-5 городов по количеству проживающих:
city
д. Сорочинск      2
д. Россошь        1
клх Мичуринск     1
п. Вологда        1
клх Костомукша    1
Name: count, dtype: int64


Преобразование в DataFrame для визуализации

In [7]:
top_cities_df = top_cities.reset_index()
top_cities_df.columns = ["city", "population"]

Топ 5 городов по численности.

In [13]:
bar_cities = alt.Chart(top_cities_df).mark_bar().encode(
    x=alt.X('city:N', title="Город"),
    y=alt.Y('population:Q', title="Количество людей"),
    color=alt.condition(  # Добавляем выделение столбцов при наведении
        alt.datum.population == top_cities_df['population'].max(),  # Условие для выделения
        alt.value('orange'),  # Цвет для выделения
        alt.value('steelblue')  # Цвет по умолчанию
    ),
    tooltip=[alt.Tooltip('city:N', title='Город'),  # Всплывающая подсказка
             alt.Tooltip('population:Q', title='Количество людей')]
).properties(
    title="Топ-5 городов по количеству проживающих"
)

bar_cities

alt.Chart(...)

Для построенния линейного графика количества регистраций по месяцам нужно изменить тип столбца с датой на дату, давайте посмотрим на типы столбцов, заодно изменим и другие типы данных.

In [14]:
df.dtypes

,0
id,object
name,object
age,int64
address,object
email,object
phone number,object
registration date,object
created at,object
updated at,object
deleted at,float64


In [15]:
df["registration date"] = pd.to_datetime(df["registration date"])
df["created at"] = pd.to_datetime(df["created at"])
df["updated at"] = pd.to_datetime(df["updated at"])
df["deleted at"] = pd.to_datetime(df["deleted at"])

In [16]:
df.dtypes

,0
id,object
name,object
age,int64
address,object
email,object
phone number,object
registration date,"datetime64[ns, UTC+03:00]"
created at,"datetime64[ns, UTC+03:00]"
updated at,"datetime64[ns, UTC+03:00]"
deleted at,datetime64[ns]


Далее необходимо извлечь месяц и год из даты.

In [19]:
df["registration_month"] = df["registration date"].dt.to_period("M")

df.head(1)

<ipython-input-19-39b209f66645>:1: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["registration_month"] = df["registration date"].dt.to_period("M")


,id,name,age,address,email,phone number,registration date,created at,updated at,deleted at,city,registration_month
0,dc544e1a-f9b4-4639-86ec-7585cbafdc41,Михайлова Светлана Станиславовна,73,"клх Мичуринск, ул. Коллективная, д. 2/3 стр. 9...",gerasimovdementi@example.net,8 017 573 61 01,2024-08-30 05:11:52.004000+03:00,2024-07-22 16:58:52.602000+03:00,2025-03-19 12:40:14.737000+03:00,NaT,клх Мичуринск,2024-08


Группировка по месяцам и подсчёт количества регистраций

In [20]:
registrations_by_month = df["registration_month"].value_counts().sort_index()
registrations_by_month

,count
registration_month,
2024-03,2
2024-04,8
2024-05,9
2024-06,8
2024-07,6
2024-08,6
2024-09,6
2024-10,11
2024-11,12


Преобразование в DataFrame

In [21]:
registrations_by_month_df = registrations_by_month.reset_index()
registrations_by_month_df.columns = ["month", "registrations"]

Преобразование месяца обратно в строку (для корректного отображения в Altair)

In [22]:
registrations_by_month_df["month"] = registrations_by_month_df["month"].astype(str)

In [23]:
registrations_by_month_df.head(1)

,month,registrations
0,2024-03,2


Линейный график количества регистраций по месяцам с интерактивностью

In [24]:
line_registrations = alt.Chart(registrations_by_month_df).mark_line(point=True).encode(
    x=alt.X('month:T', title="Месяц"),
    y=alt.Y('registrations:Q', title="Количество регистраций"),
    tooltip=[alt.Tooltip('month:T', title='Месяц'),  # Всплывающая подсказка
             alt.Tooltip('registrations:Q', title='Регистрации')]
).properties(
    title="Количество регистраций по месяцам"
).interactive()  # Добавляем интерактивность

line_registrations

alt.Chart(...)

In [30]:
df_2 = pd.read_excel('2.xlsx')

Топ-10 городов с наибольшим количеством людей.

In [31]:
top_10_cities = df_2.sort_values(by="name", ascending=False).head(10)
print(top_10_cities)

                    city  name
24          д. Сорочинск     2
1               г. Бреды     1
0             г. Амдерма     1
2   г. Быково (метеост.)     1
3          г. Волгодонск     1
5              г. Кириши     1
4             г. Камышин     1
7            г. Петухово     1
8             г. Плесецк     1
9               г. Уэлен     1


Гистограмма распределения количества людей

In [32]:
histogram = alt.Chart(df_2).mark_bar().encode(
    x=alt.X("name:Q", bin=True, title="Количество людей"),
    y=alt.Y("count()", title="Количество городов"),
    tooltip=[alt.Tooltip("name:Q", title="Количество людей"), alt.Tooltip("count()", title="Количество городов")]
).properties(
    title="Распределение количества людей по городам"
)

histogram

alt.Chart(...)

Видно, что только в одном городе нашлось 2 человека, в остальных по 1. Так выполнилась генерация.

Диаграмма топ-10 городов по количеству проживающих

In [33]:
top_10_chart = alt.Chart(top_10_cities).mark_bar().encode(
    x=alt.X("city:N", title="Город", sort="-y"),  # Сортировка по убыванию
    y=alt.Y("name:Q", title="Количество людей"),
    tooltip=[alt.Tooltip("city:N", title="Город"), alt.Tooltip("name:Q", title="Количество людей")]
).properties(
    title="Топ-10 городов по количеству проживающих"
)

top_10_chart

alt.Chart(...)

Линейный график, показывающий минимальное и максимальное количество людей в городах.

Для начала был выполнен расчёт минимального и максимального количества людей

In [45]:
# Сортируем по количеству людей
sorted_df = df_2.sort_values(by="name")

# Город с минимальным количеством людей
min_city = sorted_df.iloc[0]
print(f"Город с минимальным количеством людей: {min_city['city']} ({min_city['name']})")

Город с минимальным количеством людей: г. Амдерма (1)


In [46]:
# Город с максимальным количеством людей
max_city = sorted_df.iloc[-1]
print(f"Город с максимальным количеством людей: {max_city['city']} ({max_city['name']})")

Город с максимальным количеством людей: д. Сорочинск (2)


Создаём DataFrame с минимальным и максимальным количеством людей

In [48]:
min_max_df = pd.DataFrame({
    "city": [min_city["city"], max_city["city"]],
    "name": [min_city["name"], max_city["name"]]
})

min_max_df

,city,name
0,г. Амдерма,1
1,д. Сорочинск,2


In [49]:
line_chart = alt.Chart(min_max_df).mark_line(point=True).encode(
    x=alt.X("city:N", title="Город"),
    y=alt.Y("name:Q", title="Количество людей"),
    tooltip=[alt.Tooltip("city:N", title="Город"), alt.Tooltip("name:Q", title="Количество людей")]
).properties(
    title="Минимальное и максимальное количество людей в городах"
)

line_chart

alt.Chart(...)